In [4]:
# Imports

from numpy.random import seed
from random import random
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import arff

print('Import done.')

Import done.


In [5]:
# Data (weather)

datafile = arff.load(open('weather.arff'))
weather = pd.DataFrame(datafile['data'])
# weather = weather.drop(columns=[1,2])

weather[0] = weather[0].map(dict(sunny=0, overcast=1, rainy=2)).astype(float)
weather[3] = weather[3].map(dict(TRUE=1, FALSE=0)).astype(float)
weather[4] = weather[4].map(dict(yes=1, no=0)).astype(float)

x = weather.drop(columns=[4])
y = weather[4]

print(x)
print()
print(list(y))

TypeError: 'generator' object is not subscriptable

In [6]:
# Data (tennis)

weather = pd.read_csv('tennis.csv')

weather.outlook = weather.outlook.map(dict(sunny=0, overcast=1, rainy=2)).astype(float)
weather.temp = weather.temp.map(dict(hot=0, mild=1, cool=2)).astype(float)
weather.humidity = weather.humidity.map(dict(normal=0, high=1)).astype(float)
weather.windy = weather.windy.astype(float)
weather.play = weather.play.map(dict(yes=1, no=0)).astype(float)

x = weather.drop(columns=['play'])
y = weather.play

print(x)
print()
print(list(y))

    outlook  temp  humidity  windy
0       0.0   0.0       1.0    0.0
1       0.0   0.0       1.0    1.0
2       1.0   0.0       1.0    0.0
3       2.0   1.0       1.0    0.0
4       2.0   2.0       0.0    0.0
5       2.0   2.0       0.0    1.0
6       1.0   2.0       0.0    1.0
7       0.0   1.0       1.0    0.0
8       0.0   2.0       0.0    0.0
9       2.0   1.0       0.0    0.0
10      0.0   1.0       0.0    1.0
11      1.0   1.0       1.0    1.0
12      1.0   0.0       0.0    0.0
13      2.0   1.0       1.0    1.0

[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]


In [9]:
# Split data

X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=0)
X_validation, X_testing, y_validation, y_testing = train_test_split(X_test,
                                                                   y_test,
                                                                   test_size=0.5,
                                                                   random_state=0)

X_train = np.asarray(X_train)
X_validation = np.asarray(X_validation)
X_testing = np.asarray(X_testing)
y_train = np.asarray(y_train)
y_validation = np.asarray(y_validation)
y_testing = np.asarray(y_testing)

print('X_train')
print(X_train)
print()

print('y_train')
print(y_train)
print()

print('X_validation')
print(X_validation)
print()

print('y_validation')
print(y_validation)
print()

print('X_testing')
print(X_testing)
print()

print('y_testing')
print(y_testing)
print()

X_train
[[1. 1. 1. 1.]
 [1. 0. 1. 0.]
 [2. 1. 1. 1.]
 [2. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [2. 1. 1. 0.]
 [0. 0. 1. 0.]
 [2. 2. 0. 1.]
 [1. 0. 0. 0.]]

y_train
[1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]

X_validation
[[0. 2. 0. 0.]]

y_validation
[1.]

X_testing
[[2. 2. 0. 0.]
 [1. 2. 0. 1.]]

y_testing
[1. 1.]



In [10]:
# Backpropagation class

class BackPropagation:
    n_inputs = 1
    n_hiddens = [1]
    n_outputs = 1
    l_rate = 0.5
    n_epoch = 20
    batch_size = 1
    momentum = 1
    network = list()
    
    def __init__(self, n_inputs, n_hiddens, l_rate, n_epoch, batch_size, momentum):
        self.n_inputs = n_inputs
        self.n_hiddens = n_hiddens
        self.l_rate = l_rate
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.momentum = momentum
        self.network = list()

        hidden_layer = [{'weights':[random() for i in range(self.n_inputs + 1)]} for i in range(self.n_hiddens[0])]
        self.network.append(hidden_layer)

        for i in range(1, min(len(n_hiddens), 10)):
            hidden_layer = [{'weights':[random() for j in range(self.n_hiddens[i-1] + 1)]} for i in range(self.n_hiddens[i])]
            self.network.append(hidden_layer)

        output_layer = [{'weights':[random() for i in range(self.n_hiddens[-1] + 1)]} for i in range(self.n_outputs)]
        self.network.append(output_layer)
        
    def activate(self, weights, inputs):
        activation = weights[-1]
        for i in range(len(weights)-1):
            activation += weights[i] * inputs[i]
        return activation

    def transfer(self, activation):
        return 1.0 / (1.0 + np.exp(-activation))

    def forward_propagate(self, network, init_input):
        inputs = init_input
        for layer in network:
            new_inputs = []
            for neuron in layer:
                activation = self.activate(neuron['weights'], inputs)
                neuron['output'] = self.transfer(activation)
                new_inputs.append(neuron['output'])
            inputs = new_inputs
        return inputs
    
    def transfer_derivative(self, output):
        return output * (1.0 - output)

    def backward_propagate_error(self, network, expected):
        for i in reversed(range(len(network))):
            layer = network[i]
            errors = list()
            if i != len(network)-1:
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in network[i + 1]:
                        error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
            else:
                for j in range(len(layer)):
                    neuron = layer[j]
                    a = 1
                    errors.append(expected[j] - neuron['output'])
            for j in range(len(layer)):
                neuron = layer[j]
                neuron['delta'] = errors[j] * self.transfer_derivative(neuron['output'])
    
    def initialize_delta_weight(self, network):
        delta_weight = []
        for layer in network:
            delta_weight.append([])
            for neuron in layer:
                delta_weight[-1].append(np.zeros(len(neuron['weights'])))
        return delta_weight

    def accumulate_delta_weight(self, network, init_input, l_rate, momentum, delta_weight):
        for i in range(len(network)):
            inputs = init_input[:-1]
            if i != 0:
                inputs = [neuron['output'] for neuron in network[i - 1]]
            for j in range(len(network[i])):
                for k in range(len(inputs)):
                    delta_weight[i][j][k] += (l_rate * network[i][j]['delta'] * inputs[k] + momentum * network[i][j]['weights'][k]) 
                delta_weight[i][j][-1] += (l_rate * network[i][j]['delta'] + momentum * network[i][j]['weights'][-1]) 

    def update_weight(self, network, delta_weight):
        for i in range(len(network)):
            for j in range(len(network[i])):
                for k in range(len(network[i][j]['weights'])):
                    network[i][j]['weights'][k] += delta_weight[i][j][k]

    def fit(self, init_inputs_x, init_inputs_y, validation_data = None):
        init_inputs = list()
        for i in range(0,len(init_inputs_x)):
            init_inputs.append(np.append(init_inputs_x[i],init_inputs_y[i]))
        for epoch in range(self.n_epoch):
            sum_error = 0
            batch_member_calculated = 0
            delta_weight = self.initialize_delta_weight(self.network)
            for init_data in init_inputs:
                outputs = self.forward_propagate(self.network, init_data)
                expected = [0 for i in range(self.n_outputs)]
                expected[0] = init_data[-1]
                sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
                self.backward_propagate_error(self.network, expected)

                self.accumulate_delta_weight(self.network, init_data, self.l_rate, self.momentum, delta_weight)
                batch_member_calculated += 1

                if batch_member_calculated == self.batch_size:
                    self.update_weight(self.network, delta_weight)
                    batch_member_calculated = 0
                    delta_weight = self.initialize_delta_weight(self.network)
            if validation_data != None:
                pred = self.predict(validation_data[0])
                val_score = accuracy_score(pred, validation_data[1])
                print('>epoch=%d, lrate=%.3f, error=%.3f, val_score=%.2f, outputs=%.2f' % (epoch, self.l_rate, sum_error, val_score, outputs[0]))
            else:
                print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, self.l_rate, sum_error))
        tempnetwork = list()
        for layer in self.network:
            templayer = list()
            for neuron in layer:
                templayer.append({'weights' : neuron['weights']})
            tempnetwork.append(templayer)
        self.network = tempnetwork
            
    def predict(self, init_inputs):
        labels = np.zeros(len(init_inputs))
        for i in range(0,len(init_inputs)):
            labels[i] = self.forward_propagate(self.network, init_inputs[i])[0].round()
        return labels

In [16]:
# Backpropagation training

seed(1)
tp = (X_validation, y_validation)
bp = BackPropagation(n_inputs=4, n_hiddens=[2], l_rate=0.5, n_epoch=20, batch_size=1, momentum=0.001)
bp.fit(X_train, y_train, tp)
for layer in bp.network:
    for neuron in layer:
        print(neuron)
print()
print(bp.predict(X_testing))

>epoch=0, lrate=0.500, error=3.555, val_score=1.00, outputs=0.70
>epoch=1, lrate=0.500, error=3.201, val_score=1.00, outputs=0.61
>epoch=2, lrate=0.500, error=3.028, val_score=1.00, outputs=0.54
>epoch=3, lrate=0.500, error=2.986, val_score=1.00, outputs=0.51
>epoch=4, lrate=0.500, error=2.982, val_score=1.00, outputs=0.49
>epoch=5, lrate=0.500, error=2.983, val_score=1.00, outputs=0.49
>epoch=6, lrate=0.500, error=2.984, val_score=1.00, outputs=0.48
>epoch=7, lrate=0.500, error=2.984, val_score=1.00, outputs=0.48
>epoch=8, lrate=0.500, error=2.983, val_score=1.00, outputs=0.48
>epoch=9, lrate=0.500, error=2.982, val_score=1.00, outputs=0.48
>epoch=10, lrate=0.500, error=2.981, val_score=1.00, outputs=0.48
>epoch=11, lrate=0.500, error=2.980, val_score=1.00, outputs=0.48
>epoch=12, lrate=0.500, error=2.979, val_score=1.00, outputs=0.48
>epoch=13, lrate=0.500, error=2.978, val_score=1.00, outputs=0.48
>epoch=14, lrate=0.500, error=2.978, val_score=1.00, outputs=0.48
>epoch=15, lrate=0.5

In [18]:
# Keras training

seed(1)
model = Sequential()
model.add(Dense(2, input_dim=4, activation="sigmoid", bias_initializer="ones"))
model.add(Dense(1))
model.add(Activation("sigmoid"))

sgd = optimizers.SGD(lr=0.7, momentum=0)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=1, epochs=20, validation_data=(X_validation, y_validation))
model.predict(X_testing, batch_size=1)

Train on 11 samples, validate on 1 samples
Epoch 1/20
11/11 [==============================] - 1s 54ms/step - loss: 0.7862 - acc: 0.4545 - val_loss: 1.4747 - val_acc: 0.0000e+00
Epoch 2/20
11/11 [==============================] - 0s 2ms/step - loss: 0.9752 - acc: 0.2727 - val_loss: 0.6106 - val_acc: 1.0000
Epoch 3/20
11/11 [==============================] - 0s 4ms/step - loss: 0.9164 - acc: 0.1818 - val_loss: 0.4634 - val_acc: 1.0000
Epoch 4/20
11/11 [==============================] - 0s 2ms/step - loss: 0.8576 - acc: 0.5455 - val_loss: 1.3133 - val_acc: 0.0000e+00
Epoch 5/20
11/11 [==============================] - 0s 7ms/step - loss: 0.8662 - acc: 0.4545 - val_loss: 0.3870 - val_acc: 1.0000
Epoch 6/20
11/11 [==============================] - 0s 5ms/step - loss: 0.8612 - acc: 0.3636 - val_loss: 0.9877 - val_acc: 0.0000e+00
Epoch 7/20
11/11 [==============================] - 0s 1ms/step - loss: 0.8498 - acc: 0.3636 - val_loss: 0.4580 - val_acc: 1.0000
Epoch 8/20
11/11 [================

array([[0.6421593 ],
       [0.48254406]], dtype=float32)